In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import (VGG16, VGG19, ResNet50, InceptionV3, MobileNetV2,
                                DenseNet121, Xception)
from keras.applications.efficientnet import (EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                             EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                             EfficientNetB6, EfficientNetB7, preprocess_input as efficientnet_preprocess_input)
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocess_input
from keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess_input
from keras.applications.densenet import preprocess_input as densenet_preprocess_input
from keras.applications.xception import preprocess_input as xception_preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.layers import BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import seaborn as sns
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
# Define your data directories
train_folder = '/kaggle/input/chest-ctscan-images/Data/train'
valid_folder = '/kaggle/input/chest-ctscan-images/Data/valid'
test_folder = '/kaggle/input/chest-ctscan-images/Data/test'

In [ ]:
# Model configurations
num_classes = 4
batch_size = 32
epochs = 100

In [ ]:
# Map model names to their function, preprocess_input, and input_shape
model_info = {
    'VGG16': (VGG16, vgg16_preprocess_input, (224, 224, 3)),
    'VGG19': (VGG19, vgg19_preprocess_input, (224, 224, 3)),
    'ResNet50': (ResNet50, resnet50_preprocess_input, (224, 224, 3)),
    'InceptionV3': (InceptionV3, inceptionv3_preprocess_input, (299, 299, 3)),
    'MobileNetV2': (MobileNetV2, mobilenetv2_preprocess_input, (224, 224, 3)),
    'DenseNet121': (DenseNet121, densenet_preprocess_input, (224, 224, 3)),
    'Xception': (Xception, xception_preprocess_input, (299, 299, 3)),
    'EfficientNetB0': (EfficientNetB0, efficientnet_preprocess_input, (224, 224, 3)),
    'EfficientNetB1': (EfficientNetB1, efficientnet_preprocess_input, (240, 240, 3)),
    'EfficientNetB2': (EfficientNetB2, efficientnet_preprocess_input, (260, 260, 3)),
    'EfficientNetB3': (EfficientNetB3, efficientnet_preprocess_input, (300, 300, 3)),
    'EfficientNetB5': (EfficientNetB5, efficientnet_preprocess_input, (224, 224, 3)), 
    'EfficientNetB4': (EfficientNetB4, efficientnet_preprocess_input, (224, 224, 3)),
    'EfficientNetB6': (EfficientNetB6, efficientnet_preprocess_input, (300, 300, 3)),
    'EfficientNetB7': (EfficientNetB7, efficientnet_preprocess_input, (300, 300, 3)),
}


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
def train_model(model_name, num_classes, train_folder, valid_folder, epochs, batch_size):
    model_class, preprocess_input, input_shape = model_info[model_name]

    # Data generators with model-specific preprocessing
    train_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False
    )
    val_datagen = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
    )
    train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical'
    )
    validation_generator = val_datagen.flow_from_directory(
        valid_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    # Model creation
    base_model = model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define Callbacks
    checkpoint = ModelCheckpoint(filepath=f'{model_name}_model.h5',
                                 monitor='val_accuracy',
                                 mode='max',
                                 save_best_only=True,
                                 verbose=1)

    earlystop = EarlyStopping(monitor='val_accuracy',
                              min_delta=0.001,
                              patience=15,
                              restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=10,
                                  verbose=1,
                                  min_delta=0.0001,
                                  min_lr=0.0001)

    callbacks = [checkpoint, earlystop, reduce_lr]

    # Train the model
    history = model.fit(train_generator, validation_data=validation_generator, callbacks=callbacks, epochs=epochs, steps_per_epoch=np.ceil(train_generator.samples/batch_size), validation_steps=np.ceil(validation_generator.samples/batch_size))
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Save the plot
    plt.savefig(f'{model_name}_accuracy_loss_plot.png')
    plt.close()
    
    return model


In [ ]:
# Train and save each model
for model_name in model_info.keys():
    print(f"Training {model_name}...")
    model = train_model(model_name, num_classes, train_folder, valid_folder, epochs, batch_size)
#     model.save(f'{model_name}_model.h5')
    print(f"{model_name} model training completed and saved.")
    

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import tensorflow.keras
import os

# Function to evaluate the model and generate necessary metrics
def evaluate_model(model, test_generator, model_name):
    # Evaluate the model
    loss, accuracy = model.evaluate(test_generator, steps=np.ceil(test_generator.samples/test_generator.batch_size))

    # Generate predictions
    test_generator.reset()
    predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples/test_generator.batch_size))
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes

    # Compute confusion matrix and classification report
    cm = confusion_matrix(true_classes, predicted_classes)
    report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys(), output_dict=True)

    # Extract overall precision, recall, and f1-score
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    return accuracy, loss, precision, recall, f1_score, cm

# Path to save the CSV file
csv_file = 'model_results.csv'

# Check if the CSV file exists. If not, create it with appropriate headers
if not os.path.exists(csv_file):
    pd.DataFrame(columns=['Model', 'Accuracy', 'Loss', 'Precision', 'Recall', 'F1 Score']).to_csv(csv_file, index=False)

    
    # Function to plot and save confusion matrix as heatmap
def plot_and_save_confusion_matrix(cm, model_name, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.savefig(f'{model_name}_confusion_matrix.png')
    
# Evaluate each model and append results to the CSV file
for model_name in model_info.keys():
    
    model_class, preprocess_input, input_shape = model_info[model_name]
    
    test_datagen = ImageDataGenerator(
        dtype='float32',
        preprocessing_function=preprocess_input,
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False,
    )
    
    model = tensorflow.keras.models.load_model(f'{model_name}_model.h5')
    accuracy, loss, precision, recall, f1_score, cm = evaluate_model(model, test_generator, model_name)

    # Append results to the CSV
    new_row = pd.DataFrame([[model_name, accuracy, loss, precision, recall, f1_score]], columns=['Model', 'Accuracy', 'Loss', 'Precision', 'Recall', 'F1 Score'])
    pd.concat([pd.read_csv(csv_file), new_row]).to_csv(csv_file, index=False)

    # Print Confusion Matrix
    print(f'Confusion Matrix for {model_name}:')
    print(cm)
    # Plot and save Confusion Matrix as Heatmap
    plot_and_save_confusion_matrix(cm, model_name, list(test_generator.class_indices.keys()))